<a href="https://colab.research.google.com/github/kumawatmanish05/Question_Answering_Bot_Pytorch_RNN/blob/main/QA_Bot_Pytorch_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
#Tokenize

def tokenize(text):
    text = text.lower()
    text = text.replace('?','')
    text = text.replace("'","")
    return text.split()

In [3]:
tokenize('What is the capital of france?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [9]:
#Vacob
vocab = {'<UNK>':0}

In [10]:
def bulid_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token]= len(vocab)

In [11]:
df.apply(bulid_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [12]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [13]:
#Convert words to numerical indices

def text_to_indices(text,vocab):

    indexed_text = []

    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [14]:
text_to_indices("What is manish",vocab)

[1, 2, 0]

In [15]:
import torch
from torch.utils.data import Dataset,DataLoader

In [18]:
class QADataset(Dataset):

    def __init__(self, df ,vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self,index):

        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer  = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

        return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [19]:
dataset = QADataset(df,vocab)

In [20]:
dataloader = DataLoader(dataset ,batch_size = 1,shuffle = True)

In [21]:
for question,answer in dataloader:
    print(question,answer[0])

tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([6])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([276])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([149])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([116])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([205])
tensor([[ 10,  11, 157, 158, 159]]) tensor([160])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([244])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([173])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([99])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([85])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([156])
tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor

In [22]:
import torch.nn as nn

In [26]:
class SimpleRNN(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim = 50)
        self.rnn = nn.RNN(50,64,batch_first = True)
        self.fc = nn.Linear(64,vocab_size)

    def forward(self,question):
        embedded_question = self.embedding(question)
        hidden,final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))

        return output


In [29]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [27]:
learning_rate = 0.001
epochs = 20


In [30]:
model = SimpleRNN(len(vocab))

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [35]:
#Trainig Loop

for epoch in range (epochs):

    total_loss = 0

    for question,answer in dataloader:

        optimizer.zero_grad()

        #Forward Pass
        output = model(question)

        #Loss ->Output shape(1,324)-(1)
        loss = criterion(output,answer[0])

        #Backward Pass
        loss.backward()

        #Update
        optimizer.step()

        total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/20 | Loss: 0.0678
Epoch 1/20 | Loss: 0.1332
Epoch 1/20 | Loss: 0.1965
Epoch 1/20 | Loss: 0.2589
Epoch 1/20 | Loss: 0.3227
Epoch 1/20 | Loss: 0.3857
Epoch 1/20 | Loss: 0.4536
Epoch 1/20 | Loss: 0.5150
Epoch 1/20 | Loss: 0.5763
Epoch 1/20 | Loss: 0.6392
Epoch 1/20 | Loss: 0.7070
Epoch 1/20 | Loss: 0.7704
Epoch 1/20 | Loss: 0.8344
Epoch 1/20 | Loss: 0.8987
Epoch 1/20 | Loss: 0.9652
Epoch 1/20 | Loss: 1.0278
Epoch 1/20 | Loss: 1.0929
Epoch 1/20 | Loss: 1.1583
Epoch 1/20 | Loss: 1.2215
Epoch 1/20 | Loss: 1.2837
Epoch 1/20 | Loss: 1.3518
Epoch 1/20 | Loss: 1.4148
Epoch 1/20 | Loss: 1.4817
Epoch 1/20 | Loss: 1.5426
Epoch 1/20 | Loss: 1.6028
Epoch 1/20 | Loss: 1.6639
Epoch 1/20 | Loss: 1.7284
Epoch 1/20 | Loss: 1.7935
Epoch 1/20 | Loss: 1.8576
Epoch 1/20 | Loss: 1.9244
Epoch 1/20 | Loss: 1.9900
Epoch 1/20 | Loss: 2.0542
Epoch 1/20 | Loss: 2.1176
Epoch 1/20 | Loss: 2.1827
Epoch 1/20 | Loss: 2.2456
Epoch 1/20 | Loss: 2.3095
Epoch 1/20 | Loss: 2.3696
Epoch 1/20 | Loss: 2.4285
Epoch 1/20 |

In [36]:
def predict(model,question,threshold = 0.5):

    #Convert question to numbers
    numerical_question = text_to_indices(question,vocab)

    #Tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    #Prediction
    output = model(question_tensor)

    #Convert logits into probability
    probs = torch.nn.functional.softmax(output,dim=1)

    #Find index of max prob
    value,index = torch.max(probs,dim=1)

    if value < threshold:
        return "I don't know"
    else:
        return(list(vocab.keys())[index])

In [37]:
predict(model,"what is the largest planet in our solar system?")

'jupiter'

In [42]:
predict(model,"What is the boiling point of water in Celsius?")

'100'